In [1]:
#Importando Bibliotecas Pandas e Numpy
import pandas as pd
import numpy as np
import os, sys
import time

#Importando BiBliotecas DASK  (alternativa ao Pandas)
import dask.dataframe as dd

#Otimizador Giovanni
from sklearnex import patch_sklearn 
patch_sklearn()


CSV_PATH = "/home/gi/Desktop/Semestre8/Inteligencia_Computacional/TP1_csvs" # Path Giovanni
# CSV_PATH = "/home/gi/Desktop/Semestre8/Inteligencia_Computacional" # Path Giovanni
# CSV_PATH = "E:/Documentos/CEFET/OneDrive/Documentos/2ECOM067_INTELIGENCIA-COMPUTACIONAL-I_T01/TP1/bases/" # Path Jorge
start_time = time.time()

import warnings
warnings.simplefilter("ignore")
#Carregando Base de dados principal
df_test  = pd.read_csv(f"{CSV_PATH}/df_test.csv")
df_train = pd.read_csv(f"{CSV_PATH}/df_train_full.csv")


Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
df_train = df_train.sample(frac=0.1, random_state=35) # Amostrando 10% da base de treino
df_train.info(max_cols=1000)

<class 'pandas.core.frame.DataFrame'>
Index: 30751 entries, 3785 to 205043
Data columns (total 195 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    TARGET                        30751 non-null  int64  
 1    CODE_GENDER                   30751 non-null  object 
 2    FLAG_OWN_CAR                  30751 non-null  object 
 3    FLAG_OWN_REALTY               30751 non-null  object 
 4    CNT_CHILDREN                  30751 non-null  int64  
 5    AMT_INCOME_TOTAL              30751 non-null  float64
 6    NAME_INCOME_TYPE              30751 non-null  object 
 7    NAME_EDUCATION_TYPE           30751 non-null  object 
 8    NAME_FAMILY_STATUS            30751 non-null  object 
 9    NAME_HOUSING_TYPE             30751 non-null  object 
 10   REGION_POPULATION_RELATIVE    30751 non-null  float64
 11   DAYS_BIRTH                    30751 non-null  int64  
 12   DAYS_EMPLOYED                 30751 non-null 

In [3]:
# Treinamento
# Importanto as Bibliotecas dos classificadores
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
modelos =   [('XGBOOST  - Config.01 - random_state=42, learning_rate = 0.3, max_depth = 6', XGBClassifier(random_state=42, learning_rate = 0.3, max_depth = 6)), 
             #('XGBOOST  - Config.02 - random_state=42, learning_rate = 0.3, max_depth = 8', XGBClassifier(random_state=42, learning_rate = 0.3, max_depth = 8)), 
             #('XGBOOST  - Config.03 - random_state=42, learning_rate = 0.2, max_depth = 6', XGBClassifier(random_state=42, learning_rate = 0.2, max_depth = 6)), 
             #('XGBOOST  - Config.04 - random_state=42, learning_rate = 0.2, max_depth = 8', XGBClassifier(random_state=42, learning_rate = 0.2, max_depth = 8)), 
             #('LIGHTGBM - Config.01 - random_state=42, learning_rate = 0.3, max_depth = 6', LGBMClassifier(random_state=42, learning_rate = 0.3, max_depth = 6, force_col_wise=True)), 
             #('LIGHTGBM - Config.02 - random_state=42, learning_rate = 0.3, max_depth = 8', LGBMClassifier(random_state=42, learning_rate = 0.3, max_depth = 8, force_col_wise=True)), 
             #('LIGHTGBM - Config.03 - random_state=42, learning_rate = 0.2, max_depth = 6', LGBMClassifier(random_state=42, learning_rate = 0.2, max_depth = 6, force_col_wise=True)), 
             #('LIGHTGBM - Config.04 - random_state=42, learning_rate = 0.2, max_depth = 8', LGBMClassifier(random_state=42, learning_rate = 0.2, max_depth = 8, force_col_wise=True)), 
             #('CATBOOST - Config.01 - random_state=42, learning_rate = 0.3, max_depth = 6', CatBoostClassifier(random_state=42, learning_rate = 0.3, max_depth = 6)), 
             #('CATBOOST - Config.02 - random_state=42, learning_rate = 0.3, max_depth = 8', CatBoostClassifier(random_state=42, learning_rate = 0.3, max_depth = 8)), 
             #('CATBOOST - Config.03 - random_state=42, learning_rate = 0.2, max_depth = 6', CatBoostClassifier(random_state=42, learning_rate = 0.2, max_depth = 6)), 
             #('CATBOOST - Config.04 - random_state=42, learning_rate = 0.2, max_depth = 8', CatBoostClassifier(random_state=42, learning_rate = 0.2, max_depth = 8)),     
            ]

atributos_categoricos = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE',
    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
    'OCCUPATION_TYPE', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
    'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE',
    'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE', 'STATUS',
    'NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
    'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
    'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE',
    'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO',
    'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY',
    'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION', 'NAME_CONTRACT_STATUS'
]

import re
X_train_local = pd.get_dummies(df_train, columns=atributos_categoricos, drop_first=True)
X_train_local.columns = [re.sub(r'[^\w]', '', col) for col in X_train_local.columns]
y_train_local = X_train_local['TARGET']
X_train_local = X_train_local.drop(columns=['TARGET'])


In [ ]:
def tem_caractere_proibido(col):
    return bool(re.search(r'[^a-zA-Z0-9_]', col))

colunas_invalidas = [col for col in X_train_local if tem_caractere_proibido(col)]
print("Colunas com caracteres inválidos:", colunas_invalidas)

Colunas com caracteres inválidos: ['NAME_INCOME_TYPE_Maternity leave', 'NAME_INCOME_TYPE_State servant', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_EDUCATION_TYPE_Incomplete higher', 'NAME_EDUCATION_TYPE_Lower secondary', 'NAME_EDUCATION_TYPE_Secondary / secondary special', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_HOUSING_TYPE_House / apartment', 'NAME_HOUSING_TYPE_Municipal apartment', 'NAME_HOUSING_TYPE_Office apartment', 'NAME_HOUSING_TYPE_Rented apartment', 'NAME_HOUSING_TYPE_With parents', 'OCCUPATION_TYPE_Cleaning staff', 'OCCUPATION_TYPE_Cooking staff', 'OCCUPATION_TYPE_Core staff', 'OCCUPATION_TYPE_HR staff', 'OCCUPATION_TYPE_High skill tech staff', 'OCCUPATION_TYPE_IT staff', 'OCCUPATION_TYPE_Low-skill Laborers', 'OCCUPATION_TYPE_Medicine staff', 'OCCUPATION_TYPE_Private service staff', 'OCCUPATION_TYPE_Realty agents', 'OCCUPATION_TYPE_Sales staff', 'OCCUPATION_TYPE_Security staff', 'OCCUPATION_TYPE_Waiters/barmen staff', 'ORGANIZATION_TYPE_Business Entity Type 1', 

In [ ]:
# Definicao de hiperparametros para o GridSearch
from sklearn.model_selection import GridSearchCV

def melhores_parametros_modelo(param_grid, modelo_base, X_train, y_train):
    """
    Funcao para encontrar os melhores hiperparametros para o modelo
    """
    grid = GridSearchCV(estimator=modelo_base, param_grid=param_grid,
                    scoring='roc_auc', cv=10, n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    print(f"Melhores hiperparametros: {grid.best_params_}, ROC AUC: {grid.best_score_}")

In [ ]:
# Definicao de hiperparametros para o GridSearch
from sklearn.model_selection import GridSearchCV

# Define os parâmetros a serem testados para cada modelo
param_grid_xgb = {
    'max_depth': [4, 5, 6],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [100, 150, 200],
    'subsample': [0.8, 1.0]
}

param_grid_lgbm = {
    'max_depth': [4, 6],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [100, 150, 200],
    'subsample': [0.8, 1.0]
}

param_grid_catboost = {
    'depth': [4, 6],
    'learning_rate': [0.1, 0.2, 0.3],
    'iterations': [100, 150, 200],
    'l2_leaf_reg': [1, 3]
}

# Cria o classificador base
modelo_xgb = XGBClassifier(random_state=42)
modelo_lgbm = LGBMClassifier(random_state=42, force_col_wise=True)
modelo_catboost = CatBoostClassifier(random_state=42, verbose=0)

In [ ]:
melhores_parametros_modelo(param_grid_xgb, modelo_xgb, X_train_local, y_train_local)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Melhores hiperparametros: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.8}, ROC AUC: 0.7291956437034702


In [16]:
# Aplica o GridSearch nos modelos
# melhores_parametros_modelo(param_grid_xgb, modelo_xgb, X_train_local, y_train_local)
melhores_parametros_modelo(param_grid_lgbm, modelo_lgbm, X_train_local, y_train_local)
melhores_parametros_modelo(param_grid_catboost, modelo_catboost, X_train_local, y_train_local)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[LightGBM] [Info] Number of positive: 2032, number of negative: 22568
[LightGBM] [Info] Total Bins 20057
[LightGBM] [Info] Number of data points in the train set: 24600, number of used features: 359
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082602 -> initscore=-2.407512
[LightGBM] [Info] Start training from score -2.407512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 2032, number of negative: 22568
[LightGBM] [Info] Number of positive: 2033, number of negative: 22568
[LightGBM] [Info] Number of positive: 2033, number of negative: 22568
[LightGBM] [Info] Total Bins 20057
[LightGBM] [Info] Number of data points in the train set: 24600, number of used features: 359
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082602 -> initscore=-2.407512
[LightGBM] [Info] Start training from score -2.407512
[LightGBM] [Info] Number of positive: 2033, number of neg

In [12]:
# Combinacao com Voting Classifier
# Importando Bibliotecas pertinentes
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Fazendo validação cruzada (manter a proporção de classes)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [9]:
def avaliar_modelo_teste(modelos, X, y, nome="Modelo único de votação"): # Modelo único
    roc_aucs = []
    for train_idx, test_idx in skf.split(X, y):
        soma_preds = np.zeros(len(test_idx))

        for modelo in modelos:
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            modelo.fit(X_train, y_train)
            preds = modelo.predict(X_test)

            soma_preds += preds  # soma as previsões

        # Votação: se 3 ou mais classificadores disseram 1 → classe 1
        preds_final = [1 if s > 2 else 0 for s in soma_preds]

        # Calcula AUC para esse fold
        roc_aucs.append(roc_auc_score(y_test, preds_final))

    print(f"{nome}: ROC AUC = {np.mean(roc_aucs):.4f} ± {np.std(roc_aucs):.4f}")

In [13]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Define os parâmetros a serem testados

xgboost_params = {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.8}
lgbm_params = {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.8}
catboost_params = {'depth': 4, 'iterations': 150, 'l2_leaf_reg': 3, 'learning_rate': 0.1}

# Cria o classificador base
xgb = XGBClassifier(learning_rate=0.1,max_depth=4,n_estimators=100,subsample=0.8,random_state=42, use_label_encoder=False, eval_metric='logloss')
lgbm = LGBMClassifier(learning_rate=0.1,max_depth=4,n_estimators=100,subsample=0.8,random_state=42, force_col_wise=True)
catboost = CatBoostClassifier(depth=4, iterations=150, l2_leaf_reg=3, learning_rate=0.1, random_state=42, verbose=0)

avaliar_modelo_teste([xgb, lgbm, catboost], X_train_local, y_train_local, "Modelo único de votação")

[LightGBM] [Info] Number of positive: 2286, number of negative: 25389
[LightGBM] [Info] Total Bins 20166
[LightGBM] [Info] Number of data points in the train set: 27675, number of used features: 362
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082602 -> initscore=-2.407512
[LightGBM] [Info] Start training from score -2.407512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [16]:
def avaliar_modelo_individual(modelo, X_train_local, y_train_local, nome="Modelo único de votação"):
    """
    Funcao para avaliar o modelo individualmente
    """
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_aucs = []
    for train_idx, test_idx in skf.split(X_train_local, y_train_local):
        X_train, X_test = X_train_local.iloc[train_idx], X_train_local.iloc[test_idx]
        y_train, y_test = y_train_local.iloc[train_idx], y_train_local.iloc[test_idx]

        modelo.fit(X_train, y_train)
        preds = modelo.predict(X_test)

        # Calcula AUC para esse fold
        roc_aucs.append(roc_auc_score(y_test, preds))

    print(f"{nome}: ROC AUC = {np.mean(roc_aucs):.4f} ± {np.std(roc_aucs):.4f}")

In [18]:
# Teste com voting classifier
voting_clf = VotingClassifier(estimators=[
    ('xgb', xgb),
    ('lgbm', lgbm),
    ('catboost', catboost)
], voting='hard')
avaliar_modelo_individual(voting_clf, X_train_local, y_train_local, "Voting Classifier")

[LightGBM] [Info] Number of positive: 2286, number of negative: 25389
[LightGBM] [Info] Total Bins 20166
[LightGBM] [Info] Number of data points in the train set: 27675, number of used features: 362
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082602 -> initscore=-2.407512
[LightGBM] [Info] Start training from score -2.407512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [19]:
# Testando com Stacking classifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

stacking = StackingClassifier(
    estimators=[
        ('xgb', xgb),
        ('lgbm', lgbm),
        ('catboost', catboost)
    ],
    final_estimator=LogisticRegression(),
    cv=10
)
avaliar_modelo_individual(stacking, X_train_local, y_train_local, "Stacking Classifier")

[LightGBM] [Info] Number of positive: 2286, number of negative: 25389
[LightGBM] [Info] Total Bins 20166
[LightGBM] [Info] Number of data points in the train set: 27675, number of used features: 362
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082602 -> initscore=-2.407512
[LightGBM] [Info] Start training from score -2.407512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai